## Get file from GBDX


In [ ]:
from gbdxtools.task import env
from gbdxtools import CatalogImage
%matplotlib inline


# 103005007CF91400 amsterdam
# 10400100407D9200 amsterdam
# 103001009673BC00 Boston
# 1030010086B08100 New Delhi
# 1030010093A66D00 Boston


catalog_id = env.inputs.get('catalog_id', '1030010093A66D00')
bbox = env.inputs.get('bbox', '-71.117, 42.379, -71.114, 42.381')

image = CatalogImage(catalog_id, band_type="MS", bbox=map(float, bbox.split(",")), pansharpen = True)


import datetime

image_id = catalog_id

# Get timestamp
time_image = image.metadata["image"]["acquisitionDate"] 

# convert timestamp to datetime timestamp format
time_image = datetime.datetime.strptime(time_image, "%Y-%m-%dT%H:%M:%S.%fZ")
    
time_image_str = time_image.strftime('%Y-%m-%d_%H:%M')

In [ ]:
name = 'Camridge_science_building'

file_name = "{}_{}_{}.tif".format(name,time_image_str,image_id)

print(file_name)

tif = image.geotiff(
    path = file_name
                 )

print('saved to file')

## Upload any file with progress bar


In [ ]:
import boto3, os
import progressbar

bucket_name = "gcw-img-data"
folder_name = "worldview"
file_name = file_name
path = folder_name + "/" + file_name
s3 = boto3.client('s3')

statinfo = os.stat(file_name)

up_progress = progressbar.progressbar.ProgressBar(maxval=statinfo.st_size)

up_progress.start()

def upload_progress(chunk):
    up_progress.update(up_progress.currval + chunk)

s3.upload_file(file_name, bucket_name, path, Callback=upload_progress)

print('uploading: ', file_name)
up_progress.finish()